In [2]:
import tensorflow as tf
import plotly as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import plotly.express as px
import copy
# from stocktrends import indicators
# from stocktrends import Renko
# from Modules.Technical_indicators import ATR, BB, RSI, ADX, MACD, renko_DF
from plotly.subplots import make_subplots
import plotly.graph_objs as go
# from Modules.Plotting import Compare_Plot, Compare_Plot_bar
import warnings
warnings.filterwarnings("ignore")



In [3]:
df = pd.read_csv(r'D:\Time_series_project\data\BTC-USD.csv')
df_time = df.set_index('Date')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480 entries, 0 to 3479
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3480 non-null   object 
 1   Open       3480 non-null   float64
 2   High       3480 non-null   float64
 3   Low        3480 non-null   float64
 4   Close      3480 non-null   float64
 5   Adj Close  3480 non-null   float64
 6   Volume     3480 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 190.4+ KB


In [4]:
# fig = px.line(df, x='Date', y='Adj Close', title="BitCoin Adj closed price")
# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=1, label="1m", step="month", stepmode="backward"),
#             dict(count=6, label="6m", step="month", stepmode="backward"),
#             dict(count=1, label="YTD", step="year", stepmode="todate"),
#             dict(count=1, label="1y", step="year", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )
# fig.show()

**MACD** (Moving Average Convergence Divergence) is a versatile indicator that provides insights into a stock's momentum and potential trading opportunities based on the relationship between short-term and long-term moving averages.

In [5]:
def MACD(DF, a=12, b=26, c=9):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    df = DF.copy()
    
    # Calculate the fast and slow exponential moving averages
    df['ma_fast'] = df['Adj Close'].ewm(span=a, min_periods=a).mean()
    df['ma_slow'] = df['Adj Close'].ewm(span=b, min_periods=b).mean()
    
    # Calculate MACD line
    df['macd'] = df['ma_fast'] - df['ma_slow']
    
    # Calculate MACD signal line
    df['macd_signal'] = df['macd'].ewm(span=9, min_periods=c).mean()
    
    # Drop intermediate columns to clean up the DataFrame
    df = df.drop(columns=['ma_fast', 'ma_slow', 'macd'])
    
    # Return the DataFrame with MACD and its signal line
    return df

In [6]:
df = MACD(df)
df.tail(5)

,Date,Open,High,Low,Close,Adj Close,Volume,macd_signal
3475,2024-03-23,63802.722656,65976.398438,63038.492188,64062.203125,64062.203125,24738964812,3083.295524
3476,2024-03-24,64070.753906,67622.757813,63825.851563,67234.171875,67234.171875,27206630673,2814.974705
3477,2024-03-25,67234.093750,71162.593750,66414.835938,69958.812500,69958.812500,42700139523,2633.946459
3478,2024-03-26,69931.328125,71535.742188,69335.609375,69987.835938,69987.835938,36010437368,2511.581062
3479,2024-03-27,69988.289063,70681.898438,69501.421875,70208.960938,70208.960938,33876645888,2430.202517


Compare_Plot(data, X_axis, column1, column2)

In [5]:
# Compare_Plot(Df = df,X_axis =  'Date', column1 = 'Adj Close',column2 = 'macd_signal')


**Bollinger Bands** are a volatility-based indicator that dynamically adjusts to market conditions. The width of the bands expands and contracts based on the volatility of the underlying security. When the bands contract, it suggests low volatility, while widening bands indicate higher volatility.

In [7]:
def BB(Df, n=7):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    df = Df.copy()
    
    # Calculate the Middle Bollinger Band (MB) using the rolling mean of the adjusted close prices
    df['MB'] = df['Adj Close'].rolling(n).mean()
    
    # Calculate the Upper Bollinger Band (UB) and Lower Bollinger Band (LB) using the rolling standard deviation
    df['UB'] = df['MB'] + 2 * df['Adj Close'].rolling(n).std(ddof=0)
    df['LB'] = df['MB'] - 2 * df['Adj Close'].rolling(n).std(ddof=0)
    
    # Calculate the Bollinger Band width
    df['BB_width'] = df['UB'] - df['LB']
    
    # Return the modified DataFrame with Bollinger Bands and width
    return df

In [8]:
df_BB = BB(df)
df_BB.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'macd_signal', 'MB', 'UB', 'LB', 'BB_width'],
      dtype='object')

In [7]:
# fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
# fig.add_trace(go.Scatter(x=df_BB['Date'], y=df_BB['Adj Close'], mode='lines', name=df_BB['Adj Close'].name), row=1, col=1)
# fig.add_trace(go.Scatter(x=df_BB['Date'], y=df_BB['MB'], mode='lines', name=df_BB['MB'].name),row=1, col=1)
# fig.add_trace(go.Scatter(x=df_BB['Date'], y=df_BB['UB'], mode='lines', name=df_BB['UB'].name),row=1, col=1)
# fig.add_trace(go.Scatter(x=df_BB['Date'], y=df_BB['LB'], mode='lines', name=df_BB['LB'].name),row=1, col=1)

# fig.add_trace(go.Scatter(x=df_BB['Date'], y=df_BB['BB_width'], mode='lines', name=df_BB['BB_width'].name), row=2, col=1)
#     # Update layout to include a single slider below both subplots
# fig.update_layout(title = 'Bolinger Band Technical indicator',xaxis2=dict(rangeslider=dict(visible=True)), height=800)
# fig.update_xaxes(
#     rangeselector=dict(
#         buttons=list([
#             dict(count=1, label="1m", step="month", stepmode="backward"),
#             dict(count=6, label="6m", step="month", stepmode="backward"),
#             dict(count=1, label="YTD", step="year", stepmode="todate"),
#             dict(count=1, label="1y", step="year", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )
# fig.show()




In [9]:
df = BB(df)

**ATR (Average True Range)** is a valuable tool for traders to assess volatility, manage risk, and make informed trading decisions. It provides a quantitative measure of price movement that helps traders navigate dynamic market conditions.

In [10]:
def ATR(Df, n=7):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    df = Df.copy()
    
    # Calculate the True Range (TR) using high, low, and previous close prices
    df['C_H - C_L'] = df['High'] - df['Low']
    df['C_H - P_C'] = df['High'] - df['Adj Close'].shift(1)
    df['C_L - P_C'] = df['Low'] - df['Adj Close'].shift(1)
    df['TR'] = df[['C_H - C_L', 'C_H - P_C', 'C_L - P_C']].max(axis=1, skipna=False)
    
    # Calculate the Exponential Moving Average (EMA) of the True Range to get the Average True Range (ATR)
    df['ATR'] = df['TR'].ewm(span=n, min_periods=n).mean()
    
    # Drop the intermediate columns used to calculate TR
    df = df.drop(columns=['C_H - C_L', 'C_H - P_C', 'C_L - P_C', 'TR'])
    # Return the modified DataFrame with ATR values
    return df

In [11]:
df = ATR(df)

In [10]:
# Compare_Plot(Df = df,X_axis =  'Date', column1 = 'Adj Close',column2 = 'ATR')

**RSI (Relative Strength Index)** is a versatile tool that provides insights into price momentum, trend strength, and potential reversal points. It is commonly used by traders to make informed decisions about entry and exit points in the market. However, like any technical indicator, RSI has limitations and should be used in conjunction with other analysis techniques for comprehensive market analysis.        
The resulting RSI value oscillates between 0 and 100. A high RSI value indicates that the asset may be overbought, suggesting a potential reversal or correction in price. Conversely, a low RSI value indicates that the asset may be oversold, indicating a potential buying opportunity.

In [12]:
def RSI(Df, n=7):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    df = Df.copy()
    
    # Calculate the price change from the previous day
    df['change'] = df['Adj Close'] - df['Adj Close'].shift(1)
    
    # Calculate gains and losses
    df["gain"] = np.where(df['change'] >= 0, df['change'], 0)
    df["loss"] = np.where(df['change'] <= 0, -1 * df['change'], 0)
    
    # Calculate the exponential moving average of gains and losses
    df['avg_gain'] = df['gain'].ewm(alpha=1/n, min_periods=n).mean()
    df['avg_loss'] = df['loss'].ewm(alpha=1/n, min_periods=n).mean()
    
    # Calculate the relative strength (RS)
    df['rs'] = df['avg_gain'] / df['avg_loss']
    
    # Calculate the Relative Strength Index (RSI)
    df['RSI'] = 100 - (100 / (1 + df['rs']))
    
    # Return the RSI values
    return df['RSI']

In [13]:
df['RSI'] = RSI(df)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'macd_signal', 'MB', 'UB', 'LB', 'BB_width', 'ATR', 'RSI'],
      dtype='object')

In [12]:
Compare_Plot(Df= df,X_axis =  'Date', column1 = 'Adj Close',column2 = 'RSI')

**Average Directional Index (ADX)** is a valuable tool for traders to assess the strength and direction of trends in the market. It helps traders make informed decisions about entering or exiting trades, managing risk, and identifying potential trend reversals. However, like any technical indicator, ADX has its limitations and should be used in conjunction with other forms of analysis for comprehensive market analysis.

In [14]:
def ADX(Df, n=7):
    # Create a copy of the input DataFrame to avoid modifying the original DataFrame
    df = Df.copy()
    
    # Calculate upward and downward movements
    df['upmove'] = df['High'] - df['High'].shift()
    df['downmove'] = df['Low'].shift() - df['Low']
    
    # Calculate Positive Directional Movement (+DM) and Negative Directional Movement (-DM)
    df['+dm'] = np.where((df['upmove'] > df['downmove']) & (df['upmove'] > 0), df['upmove'], 0)
    df['-dm'] = np.where((df['downmove'] > df['upmove']) & (df['downmove'] > 0), df['downmove'], 0)
    
    # Calculate Positive Directional Indicator (+DI) and Negative Directional Indicator (-DI)
    df['+di'] = 100 * (df['+dm'] / df['ATR']).ewm(com=n, min_periods=n).mean()
    df['-di'] = 100 * (df['-dm'] / df['ATR']).ewm(com=n, min_periods=n).mean()
    
    # Calculate Directional Movement Index (DX)
    df['DX'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    
    # Calculate Average Directional Index (ADX)
    df['ADX'] = df['DX'].ewm(span=n, min_periods=n).mean()
    
    # Return the DataFrame with ADX and related components
    return df['ADX']

In [15]:
df['ADX']= ADX(df)

In [14]:
Compare_Plot(Df = df,X_axis = 'Date', column1 = 'Adj Close',column2 = 'ADX')

In [15]:
# Renko_df = renko_DF(df)
# df['date'] = df['Date']
# df = df.merge(Renko_df[['date','uptrend']], on = 'date', how = 'left')
# df['uptrend'].ffill(inplace = True)

In [16]:
# df['uptrend'] = df['uptrend'].replace({True: 1, False: -1})

In [17]:
# Compare_Plot(Df= df,X_axis =  'Date', column1 = 'Adj Close',column2 = 'uptrend')

In [18]:

# Compare_Plot_bar(Df = df,X_axis =  'Date', column1 = 'Adj Close',column2 = 'uptrend')

In [16]:
df.columns


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'macd_signal', 'MB', 'UB', 'LB', 'BB_width', 'ATR', 'RSI', 'ADX'],
      dtype='object')

In [17]:
df = df.drop(columns = ['Open', 'High', 'Low', 'Close','Volume', 'MB', 'UB', 'LB' ],axis = 1)

In [18]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3447 entries, 2014-10-20 to 2024-03-27
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Adj Close    3447 non-null   float64
 1   macd_signal  3447 non-null   float64
 2   BB_width     3447 non-null   float64
 3   ATR          3447 non-null   float64
 4   RSI          3447 non-null   float64
 5   ADX          3447 non-null   float64
dtypes: float64(6)
memory usage: 188.5 KB


In [19]:
import tensorflow as tf
import numpy as np

In [10]:
df =df.set_index('Date')

In [48]:
# labels_list = []
# feature_list = []
# window_size = 15
# horizon = 1
# feature_array = np.array(df)
# for i in range(0, len(df['Adj Close'])-(window_size+horizon-1)):
#     feature_list.append(np.array(feature_array[i:i+window_size]))
#     labels_list.append(np.array(df['Adj Close'][i+window_size:i+window_size+horizon]))

# # Convert the list of arrays to a numpy array
# labels = np.array(labels_list)
# feature = np.array(feature_list)

# labels  = tf.convert_to_tensor(labels)
# feature = tf.convert_to_tensor(feature)

In [20]:
labels_list = []
feature_list = []
window_size = 15
horizon = 7
feature_array = np.array(df)
for i in range(0, len(df['Adj Close'])-(window_size+horizon-1)):
    feature_list.append(np.array(feature_array[i:i+window_size]))
    labels_list.append(np.array(df['Adj Close'][i+window_size:i+window_size+horizon]))

# Convert the list of arrays to a numpy array
labels = np.array(labels_list)
feature = np.array(feature_list)


labels  = tf.convert_to_tensor(labels)
feature = tf.convert_to_tensor(feature)

In [21]:
def make_train_test_splits(windows, labels, test_split = 0.15):
    split_size = int(len(windows)* (1-test_split))
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]
    return train_windows, test_windows, train_labels, test_labels

In [22]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(feature,labels)
train_windows.shape, test_windows.shape, train_labels.shape, test_labels.shape

(TensorShape([2912, 15, 6]),
 TensorShape([514, 15, 6]),
 TensorShape([2912, 7]),
 TensorShape([514, 7]))

In [23]:
batch_size = 32
def prepare_data_for_training(X_train, X_test,Y_train, Y_test, batch_size = 32):

    train_text_dataset = tf.data.Dataset.from_tensor_slices(X_train)
    train_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_train)

    train_dataset = tf.data.Dataset.zip((train_text_dataset, train_labels_dataset))
    train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    test_text_dataset = tf.data.Dataset.from_tensor_slices(X_test)
    test_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_test)

    test_dataset = tf.data.Dataset.zip((test_text_dataset, test_labels_dataset))
    test_dataset = test_dataset.shuffle(buffer_size=len(X_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train_dataset, test_dataset

In [24]:
train_dataset, val_dataset = prepare_data_for_training(X_train = train_windows, X_test= test_windows,Y_train = train_labels, Y_test =  test_labels , batch_size = 32)

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor="loss",#quantity to monitor
    min_delta=0.0001,#Minimum change in the monnitored quantity as an improvement
    patience=50,#No of epochs with no improvement
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

In [26]:
import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

def create_callbacks(dir_name, experiment_name):
    """
    Creates callbacks for training.

    Args:
    - dir_name: target directory to store TensorBoard log files and model checkpoints
    - experiment_name: name of experiment directory

    Returns:
    - callbacks: list of callbacks
    """
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    print(f"Saving TensorBoard log files to: {log_dir}")

    # TensorBoard callback for logging
    tensorboard_callback = TensorBoard(log_dir=log_dir)

    # ModelCheckpoint callback for saving the best model based on training loss
    model_checkpoint_callback = ModelCheckpoint(filepath=f"{log_dir}/best_model.h5",
                                                monitor='loss',  # Monitor training loss
                                                save_best_only=True,
                                                patience=50,#No of epochs with no improvement
                                                verbose=1)

    return [tensorboard_callback, model_checkpoint_callback]


In [27]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout

# Define input layer
inputs = Input(shape=(window_size, 6))

# Define LSTM layers with dropout
lstm1 = LSTM(128, activation='relu', return_sequences=True)(inputs)
# dropout1 = Dropout(0.2)  # Add dropout after the first LSTM layer
lstm2 = LSTM(128, activation='relu', return_sequences=True)(lstm1)#(dropout1)
# dropout2 = Dropout(0.2)  # Add dropout after the second LSTM layer
lstm3 = LSTM(128, activation='relu')(lstm2)#(dropout2)

# Output layer with a single unit
outputs = Dense(horizon, activation='linear')(lstm3)

# Create model
model = Model(inputs=inputs, outputs=outputs)

# Compile model
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

# Train the model
model.fit(train_dataset,
          epochs=100,validation_data=  val_dataset, verbose=1,
          batch_size=1,callbacks = [create_callbacks('Model','model_11_multivariate_parameter'),callback ])


Saving TensorBoard log files to: Model/model_11_multivariate_parameter/20240418-184534
Epoch 1/100
91/91 [==============================] - ETA: 0s - loss: 5327.9995 - mae: 5327.9995
Epoch 1: loss improved from inf to 5327.99951, saving model to Model/model_11_multivariate_parameter/20240418-184534\best_model.h5
91/91 [==============================] - 16s 121ms/step - loss: 5327.9995 - mae: 5327.9995 - val_loss: 6251.1572 - val_mae: 6251.1572
Epoch 2/100
91/91 [==============================] - ETA: 0s - loss: 3558.1125 - mae: 3558.1125
Epoch 2: loss improved from 5327.99951 to 3558.11255, saving model to Model/model_11_multivariate_parameter/20240418-184534\best_model.h5
91/91 [==============================] - 11s 125ms/step - loss: 3558.1125 - mae: 3558.1125 - val_loss: 5987.3794 - val_mae: 5987.3794
Epoch 3/100
91/91 [==============================] - ETA: 0s - loss: 2037.0967 - mae: 2037.0967
Epoch 3: loss improved from 3558.11255 to 2037.09668, saving model to Model/model_11_mul

In [33]:
# def mean_absolute_scaled_error(y_true, y_pred, seasonality=1):
#     mae = np.mean(np.abs(y_true - y_pred))
#     naive_forecast = y_true[:-seasonality]
#     naive_mae = np.mean(np.abs(naive_forecast - y_true[seasonality:]))
#     mase = mae / naive_mae
#     return mase

# def evaluate_preds(y_true, y_pred):
#   # Make sure float32 (for metric calculations)
#   y_true = tf.cast(y_true, dtype=tf.float32)
#   y_pred = tf.cast(y_pred, dtype=tf.float32)

#   # Calculate various metrics
#   mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
#   mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
#   rmse = tf.sqrt(mse)
#   huber_loss = tf.keras.losses.Huber()(y_true, y_pred)
#   mase = mean_absolute_scaled_error(y_true.numpy(), y_pred.numpy(),seasonality=1)

#   # Account for different sized metrics (for longer horizons, reduce to single number)
#   if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
#     mae = tf.reduce_mean(mae)
#     mse = tf.reduce_mean(mse)
#     rmse = tf.reduce_mean(rmse)
#     huber_loss = tf.reduce_mean(huber_loss)
#     mase = tf.reduce_mean(mase)

#   print("For mae, mse, rmse and hubber : lower value is better.\nmase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.")
#   print(f"""
#     "mae": {mae.numpy()},
#     "mse": {mse.numpy()},
#     "rmse": {rmse.numpy()},
#     "huber_loss": {huber_loss.numpy()},
#     "mase": {mase}
#     """)
#   return {
#         "mae": mae.numpy(),
#         "mse": mse.numpy(),
#         "rmse": rmse.numpy(),
#         "huber_loss": huber_loss.numpy(),
#         "mase": mase.numpy()
#     }

In [32]:
import tensorflow as tf
import numpy as np

def mean_absolute_scaled_error(y_true, y_pred, seasonality=1):
    mae = np.mean(np.abs(y_true - y_pred))
    naive_forecast = y_true[:-seasonality]
    naive_mae = np.mean(np.abs(naive_forecast - y_true[seasonality:]))
    mase = mae / naive_mae
    return mase

def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
    rmse = tf.sqrt(mse)
    huber_loss = tf.keras.losses.Huber()(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true.numpy(), y_pred.numpy(), seasonality=1)

    # Account for different sized metrics (for longer horizons, reduce to single number)
    if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
        mae = tf.reduce_mean(mae)
        mse = tf.reduce_mean(mse)
        rmse = tf.reduce_mean(rmse)
        huber_loss = tf.reduce_mean(huber_loss)
        mase = np.mean(mase)

    print("For mae, mse, rmse and hubber : lower value is better.\nmase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.")
    print(f"""
        "mae": {mae.numpy()},
        "mse": {mse.numpy()},
        "rmse": {rmse.numpy()},
        "huber_loss": {huber_loss.numpy()},
        "mase": {mase}
        """)
    return {
        "mae": mae.numpy(),
        "mse": mse.numpy(),
        "rmse": rmse.numpy(),
        "huber_loss": huber_loss.numpy(),
        "mase": mase
    }


In [34]:
forecast  = model.predict(test_windows)
preds = forecast


17/17 [==============================] - 1s 22ms/step


In [35]:
evaluate_preds(y_true= test_labels, y_pred= preds)

For mae, mse, rmse and hubber : lower value is better.
mase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.

        "mae": 2473.06298828125,
        "mse": 10991457.0,
        "rmse": 2608.119140625,
        "huber_loss": 2472.56298828125,
        "mase": 4.418631553649902
        


{'mae': 2473.063,
 'mse': 10991457.0,
 'rmse': 2608.1191,
 'huber_loss': 2472.563,
 'mase': 4.4186316}

In [52]:
model.save('model_11_multivariate_parameter_15_7_working')

INFO:tensorflow:Assets written to: model_11_multivariate_parameter_15_7_improved\assets


In [30]:
saved_model_path = r"D:\Time_series_project\Research\Model\model_11_multivariate_parameter\20240418-184534\best_model.h5"
loaded_model = tf.keras.models.load_model(saved_model_path)

In [1]:
forecast  =loaded_model.predict(test_windows)
preds = forecast
evaluate_preds(y_true= test_labels, y_pred= preds)

NameError: name 'loaded_model' is not defined